Let's create some random data: the final grade of graduates in a class and discretize it using different ways. <br>
This is still in Python. Remember we can run plain Python.

In [0]:
import random
randomlist = []
for i in range(0, 130):
  n = random.randint(60, 100)
  randomlist.append(n)

# Appending an additional NULL for a student who was not marked
randomlist.append(None)

In [0]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import when
import pyspark.sql.functions as F
df = spark.createDataFrame(randomlist, IntegerType()).toDF("GRADE")

In [0]:
df.display()

GRADE
95
71
88
86
89
85
75
96
98
86


We want to apply the following grading scheme in the column **"LEVEL"** with the **if function** in Pyspark
- GRADE < 70%: Fail
- 70% <= GRADE < 80%: Good
- 80% <= GRADE < 90%: Very good
- GRADE >= 90%: Excellent

In [0]:
df = df.withColumn("LEVEL",
                  when(F.col('GRADE') < 70, 'D')
                   .when((F.col('GRADE') >= 70) & (F.col('GRADE') < 80), 'C')
                    .when((F.col('GRADE') >= 80) & (F.col('GRADE') < 90), 'B')
                     .when(F.col('GRADE') >= 90, 'A').otherwise('ABSENT'))

In [0]:
df.show(5)

+-----+-----+
|GRADE|LEVEL|
+-----+-----+
|   95|    A|
|   71|    C|
|   88|    B|
|   86|    B|
|   89|    B|
+-----+-----+
only showing top 5 rows



Now, let's do the same thing in a different way. Let's use some common MLlib features https://spark.apache.org/docs/latest/api/python/reference/pyspark.ml.html such as the Bucketizer https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.Bucketizer.html#pyspark.ml.feature.Bucketizer

In [0]:
from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer()

In [0]:
bucketizer.setSplits([0, 70, 80, 90, 100])
bucketizer.setInputCol("GRADE")
bucketizer.setOutputCol("BUCKET")
bucketizer.setHandleInvalid("keep") # What would you like to do with the NaN? Keep it in a separate bucket.

Out[12]: Bucketizer_7d6e0a882ef7

In [0]:
bucketed_data = bucketizer.transform(df)
bucketed_data.show(135)

+-----+------+------+
|GRADE| LEVEL|BUCKET|
+-----+------+------+
|   95|     A|   3.0|
|   71|     C|   1.0|
|   88|     B|   2.0|
|   86|     B|   2.0|
|   89|     B|   2.0|
|   85|     B|   2.0|
|   75|     C|   1.0|
|   96|     A|   3.0|
|   98|     A|   3.0|
|   86|     B|   2.0|
|   87|     B|   2.0|
|   87|     B|   2.0|
|   82|     B|   2.0|
|   93|     A|   3.0|
|   73|     C|   1.0|
|   90|     A|   3.0|
|   91|     A|   3.0|
|   86|     B|   2.0|
|   76|     C|   1.0|
|   83|     B|   2.0|
|   67|     D|   0.0|
|   86|     B|   2.0|
|   86|     B|   2.0|
|   94|     A|   3.0|
|   68|     D|   0.0|
|   86|     B|   2.0|
|   81|     B|   2.0|
|   98|     A|   3.0|
|   91|     A|   3.0|
|   90|     A|   3.0|
|   83|     B|   2.0|
|   87|     B|   2.0|
|   68|     D|   0.0|
|   86|     B|   2.0|
|   65|     D|   0.0|
|   92|     A|   3.0|
|   73|     C|   1.0|
|   72|     C|   1.0|
|   90|     A|   3.0|
|   79|     C|   1.0|
|   79|     C|   1.0|
|   97|     A|   3.0|
|   82|   

Check what happens with values 80, 90 and NaN (i.e. the last one) <br>
For variables with 5 splits, Bucketizer creates 4 Buckets. Or generally, n splits, create n-1 buckets. <br>
For continuous variables, if you don't know the minimum/maximum, you can use **-float("inf")** or **float("inf")**

A different way to bin a continuous variable is the QuantileDiscretizer https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.QuantileDiscretizer.html#pyspark.ml.feature.QuantileDiscretizer <br> Think about the percentiles in kids growth charts?

In [0]:
from pyspark.ml.feature import QuantileDiscretizer
DecileDiscretizer = QuantileDiscretizer()

In [0]:
DecileDiscretizer.setInputCol("GRADE")
DecileDiscretizer.setOutputCol("DECILE")
DecileDiscretizer.setHandleInvalid("keep")
DecileDiscretizer.setNumBuckets(10)
DecileDiscretizer.setRelativeError(0.001)

Out[22]: QuantileDiscretizer_a4148b9dc4d3

In [0]:
decile_grades = DecileDiscretizer.fit(df).transform(df)
decile_grades.show(135)

+-----+------+------+
|GRADE| LEVEL|DECILE|
+-----+------+------+
|   95|     A|   8.0|
|   71|     C|   2.0|
|   88|     B|   6.0|
|   86|     B|   5.0|
|   89|     B|   6.0|
|   85|     B|   5.0|
|   75|     C|   3.0|
|   96|     A|   8.0|
|   98|     A|   9.0|
|   86|     B|   5.0|
|   87|     B|   6.0|
|   87|     B|   6.0|
|   82|     B|   4.0|
|   93|     A|   7.0|
|   73|     C|   2.0|
|   90|     A|   7.0|
|   91|     A|   7.0|
|   86|     B|   5.0|
|   76|     C|   3.0|
|   83|     B|   4.0|
|   67|     D|   1.0|
|   86|     B|   5.0|
|   86|     B|   5.0|
|   94|     A|   8.0|
|   68|     D|   1.0|
|   86|     B|   5.0|
|   81|     B|   4.0|
|   98|     A|   9.0|
|   91|     A|   7.0|
|   90|     A|   7.0|
|   83|     B|   4.0|
|   87|     B|   6.0|
|   68|     D|   1.0|
|   86|     B|   5.0|
|   65|     D|   0.0|
|   92|     A|   7.0|
|   73|     C|   2.0|
|   72|     C|   2.0|
|   90|     A|   7.0|
|   79|     C|   3.0|
|   79|     C|   3.0|
|   97|     A|   9.0|
|   82|   

Check yourself what happens when you use the other two options for **handleInvalid**. <br> 
Option **skip** filters out any rows with NaN <br>
Option **error** throws an error if any NaN is present

In [0]:
decile_grades.groupBy("DECILE").count().show()

+------+-----+
|DECILE|count|
+------+-----+
|   8.0|   13|
|   7.0|   14|
|   4.0|   14|
|   3.0|   13|
|   2.0|   13|
|   6.0|   13|
|   5.0|   12|
|   9.0|   14|
|   1.0|   12|
|   0.0|   12|
|  null|    1|
+------+-----+



And this final groupby is a sanity check to show that all deciles have **roughly** the same number of students per decile. <br>
Why aren't all deciles equal and 13? Probably ties...